## Import libraries

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats import proportion as proptests

import matplotlib.pyplot as plt
% matplotlib inline

UsageError: Line magic function `%` not found.


## Load data

In [2]:
train_df = pd.read_csv('data/training.csv')
print('Loaded training data: size {}'.format(train_df.shape))
test_df = pd.read_csv('data/test.csv')
print('Loaded testing data: size {}'.format(test_df.shape))
print('----------------')
print('train_df:')
train_df.head()

Loaded training data: size (84534, 10)
Loaded testing data: size (41650, 10)
----------------
train_df:


,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044331,-0.385883,1,1,2,2


## Calculate number of customers and purchasers in treatment and control group

* `Treatment` group: received promotion, need to test the effect of promotion program in this group
* `Control` group: NOT received promotion

In [11]:
totalCustomer = train_df.shape[0]
numberCustomerTreatment = train_df['Promotion'].value_counts()[0]
numberCustomerControl = train_df['Promotion'].value_counts()[1]
numberPurchaserTreatment = train_df.groupby('Promotion')['purchase'].sum()[1]
numberPurchaserControl = train_df.groupby('Promotion')['purchase'].sum()[0]

print('Total customers: {}'.format(totalCustomer))
print('Number of customers in treatment group: {}'.format(numberCustomerTreatment))
print('Number of customers in control group: {}'.format(numberCustomerControl))
print('Number of purchasers in treatment group: {}'.format(numberPurchaserTreatment))
print('Number of purchasers in control group: {}'.format(numberPurchaserControl))

Total customers: 84534
Number of customers in treatment group: 42364
Number of customers in control group: 42170
Number of purchasers in treatment group: 721
Number of purchasers in control group: 319


## Calculate IRR and NIR

`IRR (Incremental Response Rate)` depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion.

$$ IRR = \frac{purchasers_{treatment}}{customers_{treatment}} - \frac{purchasers_{control}}{customers_{control}} $$


`NIR (Net Incremental Revenue)` depicts how much is made (or lost) by sending out the promotion

$$ NIR = (10\cdot purchasers_{treatment} - 0.15 \cdot customers_{treatment}) - 10 \cdot purchasers_{control}$$

In [12]:
IRR = (numberPurchaserTreatment/numberCustomerTreatment) - (numberPurchaserControl/numberCustomerControl)
print('IRR = {}'.format(IRR))

NIR = (10*numberPurchaserTreatment - 0.15*numberCustomerTreatment) - 10*numberPurchaserControl
print('NIR = {}'.format(NIR))

IRR = 0.009454547819772702
NIR = -2334.5999999999995


## Hypothesis

`Bootstrapping` method is used to test hypotheses.

By estimate sampling distributions that uses the actually collected data to generate new samples that could have been hypothetically collected. In a standard bootstrap, a bootstrapped sample means drawing points from the original data _with replacement_ until we get as many points as there were in the original data. Essentially, we're treating the original data as the population: without making assumptions about the original population distribution, using the original data as a model of the population is the best that we can do.

### Hypothesis 1

* H0: $numberCustomerTreatment = numberCustomerControl$
* Ha: $numberCustomerTreatment \ne numberCustomerControl$

$\alpha$ = 0.05 (significance level)


### Test hypothesis 1

Whether the difference between number of customers in treatment group and control group is statistically significant.

In [6]:
# ## simulate outcomes under null, compare to observed outcome
# p = 0.5
# n_trials = 200_000

# samples = np.random.binomial(totalCustomer, p, n_trials)
# p_value = np.logical_or(samples <= numberCustomerControl, samples >= numberCustomerTreatment).mean()

# print("p-value from the observed outcome: {p}".format(p=p_value))

p-value from the observed outcome: 0.50646


In [10]:
sss = int(round(train_df.shape[0] * 0.2,0))  # Sub Sample Size

prom_avg = []
non_prom_avg = []
differences = []

for _ in range(10000):
    
    sub_sample = train_df.sample(sss, replace=True)
    promotion_avg = ((sub_sample.Promotion == "Yes").sum())/sub_sample.shape[0]
    non_promotion_avg = (sub_sample.Promotion == "No").sum()/sub_sample.shape[0]
    
    prom_avg.append(promotion_avg)
    non_prom_avg.append(non_promotion_avg)
    differences.append(promotion_avg - non_promotion_avg)

null_vals = np.random.normal(0, np.std(differences), 10000)

# Determining the signifigance of our result 
p_val = (differences>null_vals).mean()
p_val

0.5819

Because `p_value = 0.5068` > $\alpha$ = 0.05, the null hypothesis is accepted. That means there's no statistical difference in the sampling population. If it's statistical significant, this will require us to revise random assignment procedures and re-do data collection. This hypothesis testing is to make sure our inferences on the desired metrics are not founded on bias due to sampling population.

### Hypothesis 2

* H0: Incremental Response Rate  = 0
* Ha: Incremental Response Rate > 0

$\alpha$ = 0.05 (significance level)



### Test hypothesis 2
Whether the promotion program had a positive effect on `IRR` metric

In [8]:
# # compute p-value
# p_null = train_df['purchase'].mean()
# se_p = np.sqrt(p_null * (1-p_null) * (1/numberCustomerControl + 1/numberCustomerTreatment))

# z_score = IRR/se_p
# p_val = 1-stats.norm.cdf(z_score)

# print("p-value: {p}".format(p=p_val))

In [9]:
# Bootstraping our differences to get a model of the distribution for our null
sub_sample_size = 5000
IRRs = []
n_trials = 10000
for _ in range(n_trials):
    bootsample2 = train_df.sample(sub_sample_size, replace=True)
    purchase_treatment = bootsample2[bootsample2['Promotion'] == "Yes"].purchase.sum()
    customer_treatment = bootsample2[bootsample2['Promotion'] == "Yes"].shape[0]
    purchase_control = bootsample2[bootsample2['Promotion'] == "No"].purchase.sum()
    customer_control = bootsample2[bootsample2['Promotion'] == "No"].shape[0]
    IRR_boot = purchase_treatment/customer_treatment - purchase_control/customer_control
    IRRs.append(IRR_boot)
    
null_IRRs = np.random.normal(sum(IRRs)/sub_sample_size, np.std(IRRs), n_trials)

# Calculating p_value
p_val_IRR = (IRR > null_IRRs).mean()
print('p_value: {}'.format(p_val_IRR))

p_value: 0.0003


### Hypothesis 3

* H0: Net Incremental Revenue = 0
* Ha: Net Incremental Revenue > 0

$\alpha$ = 0.05 (significance level)



### Test hypothesis 3
Whether the promotion program had a positive effect on `NIR` metric